In [ ]:
import sys
!{sys.executable} -m pip install openai jsonlines pyarrow pandas

# Call model with few-shot examples

In [ ]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

def callModel(prompt, prefix="", examples=[], max_tokens=1024):
    messages=[{"role": "system", "content": prefix}]

    for example in examples:
        content=f"""
{example['code']}
The solution is {example['target']}.
"""
        messages.append({"role": "user", "content": f"{example['input']}"})
        messages.append({"role": "assistant", "content": f"{content}"})

    messages.append({"role": "user", "content": f"{prompt}"})
    # print('messages: ', *messages, sep = "\n\t") 
        
    completion = client.chat.completions.create(
      model= "",
      messages=messages,
      stream=False,
      temperature=0.7,
      top_p=0.9,
      max_tokens=max_tokens,
      stop=["The solution is"]
    )
    return completion.choices[0].message.content

# Read the gsm-hard JSONL file

In [ ]:
import jsonlines
import re

file_path = '/Users/amatarazzo/Downloads/gsmhardv2.jsonl'
pattern_annotation = r'("""[^"]*""")'

# Read the JSONL file
with jsonlines.open(file_path) as reader:
    data = [{'input': obj['input'], 'code': re.sub(pattern_annotation, "", obj['code']), 'target': obj['target']} for obj in reader]
    
print(f'gsm-hard dataset loaded ({len(data)} rows)')

def extract_and_execute_function(code_string):
    # Extract the function definition using regex
    function_match = re.search(r"def\s+\w+\(.*?\):.*?(?=def|\Z)", code_string, re.DOTALL)
    
    if function_match:
        function_code = function_match.group(0)
        # Execute the function definition
        exec(function_code, globals())
        # Call the extracted function by name
        result = solution()
        return result
    else:
        return "No function found in the string"

# Run!

In [ ]:
#few-shot

import concurrent.futures
import re
import random
import math

prefix = """Your are a maths teacher. Answer the following mathematical reasoning question. 

Let's think step by step."""
right=0
wrong=0
error=0
shots=5
workers=3
max_tokens=1024

split_index=math.floor(len(data)/2)
test_data=data[:split_index]
# test_data=data[:1]
example_data=data[split_index+1:]

# Using ThreadPoolExecutor to run callModel in parallel

with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:  # Adjust max_workers as needed
    examples=random.sample(example_data, shots)
    future_to_data = {executor.submit(callModel, json_obj['input'], prefix, examples, max_tokens): json_obj for json_obj in test_data}
    for future in concurrent.futures.as_completed(future_to_data):
        json_obj = future_to_data[future]
        try:
            answer = future.result()
            match=False
            result=None
            try:
                result = extract_and_execute_function(answer)
                print(f"calculated: {result}")
                if(result==json_obj['target']):
                    match=True
            except Exception as exc:
                print(exc)
                error+=1

            print(f"""
question: {json_obj['input']}
answer:   {answer}
executed: {result}
-------------------
solution:   {json_obj['target']}
            """)
            if match:
                right+=1
                print("answer is right")
            else:
                wrong+=1
                print("answer is wrong")

            print(f"""
total: {right + wrong}
right: {right} {right*100/(right+wrong)}%
wrong: {wrong}
error: {error}
            """)
            print("===============================================================================================")

        except Exception as exc:
            print(f"An error occurred for {item}: {exc}")
            error+=1

print(f"""
total: {right + wrong}
right: {right} {right*100/(right+wrong)}%
wrong: {wrong}
error: {error}
""")